In [75]:
#import all required packages
import pandas as pd
import matplotlib.pyplot as plt
import os

import numpy as np 
from PIL import Image
from tqdm import tqdm_notebook

from sklearn.utils import shuffle
from resnets_utils import *

import imageio
from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from keras.models import model_from_json
from keras.optimizers import SGD, Adam

%matplotlib inline

In [83]:
#Initialize paths related to training and testing datasets
print("Base directory:        %s" % os.getcwd())

path =  '.'
train_folder = os.path.join(path, 'train_images')
test_folder =  os.path.join(path,  'leaderboard_test_data')
holdout_folder = os.path.join(path,  'leaderboard_holdout_data')
sample_sub = os.path.join(path, 'SampleSubmission.csv')
labels = os.path.join(path,  'traininglabels.csv')

print("Training images path:  %s " % train_folder)
print("Test images path:      %s " % test_folder)
print("Holdout images path:   %s " % holdout_folder)
print("Sample submission:     %s " % sample_sub)
print("Training Labels path:  %s " % labels)
image_size=256
image_layers=3
pixel_depth=256

Base directory:        D:\deep_udacity\deep-learning\kaggle\widsdatathon2019
Training images path:  .\train_images 
Test images path:      .\leaderboard_test_data 
Holdout images path:   .\leaderboard_holdout_data 
Sample submission:     .\SampleSubmission.csv 
Training Labels path:  .\traininglabels.csv 


In [84]:
#Method to load images available in a folder
def load_images_from_folder(folder_name):
    
    test_image_names = []
    image_files = os.listdir(folder_name)
    loaded_images = np.ndarray(shape=(len(image_files), image_size, image_size,image_layers), dtype=np.float32)

    num_images = 0

    for image_name in image_files:
 
        image_file = os.path.join(folder_name, image_name)
        try:
          image_data = (imageio.imread(image_file).astype(float) ) / pixel_depth
          loaded_images[num_images, :, :,:] = image_data
          num_images = num_images + 1
          test_image_names.append(image_name)
        
        except (IOError, ValueError) as e:
          print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')

    #If we could not read any image, resize the array to correct length
    loaded_images = loaded_images[0:num_images, :, :,:]
 
    return test_image_names, loaded_images

test_image_names, test_images = load_images_from_folder( os.path.join(path,  'leaderboard_holdout_data'))

In [85]:
print("Testing size " , str(test_images.shape))

Testing size  (2178, 256, 256, 3)


In [9]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 

Loaded model from disk


In [10]:
adam = Adam(lr=0.0001)
#loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
loaded_model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [86]:
preds = loaded_model.predict(test_images,  batch_size=16, verbose=1)

2178/2178 [==============================] - ETA: 23:1 - ETA: 23:4 - ETA: 24:0 - ETA: 24:1 - ETA: 24:3 - ETA: 24:4 - ETA: 24:5 - ETA: 25:0 - ETA: 25:5 - ETA: 26:4 - ETA: 27:2 - ETA: 27:4 - ETA: 27:2 - ETA: 27:2 - ETA: 27:3 - ETA: 27:1 - ETA: 27:0 - ETA: 27:0 - ETA: 26:5 - ETA: 26:3 - ETA: 26:2 - ETA: 26:0 - ETA: 25:4 - ETA: 25:3 - ETA: 25:1 - ETA: 24:5 - ETA: 24:4 - ETA: 24:2 - ETA: 24:1 - ETA: 23:5 - ETA: 23:4 - ETA: 23:2 - ETA: 23:1 - ETA: 22:5 - ETA: 22:4 - ETA: 22:2 - ETA: 22:1 - ETA: 22:0 - ETA: 21:4 - ETA: 21:3 - ETA: 21:2 - ETA: 21:0 - ETA: 20:5 - ETA: 20:4 - ETA: 20:2 - ETA: 20:1 - ETA: 20:0 - ETA: 19:4 - ETA: 19:3 - ETA: 19:2 - ETA: 19:0 - ETA: 18:5 - ETA: 18:3 - ETA: 18:2 - ETA: 18:1 - ETA: 18:0 - ETA: 17:5 - ETA: 17:4 - ETA: 17:3 - ETA: 17:1 - ETA: 17:0 - ETA: 16:5 - ETA: 16:4 - ETA: 16:2 - ETA: 16:1 - ETA: 16:0 - ETA: 15:4 - ETA: 15:3 - ETA: 15:2 - ETA: 15:1 - ETA: 14:5 - ETA: 14:4 - ETA: 14:3 - ETA: 14:2 - ETA: 14:0 - ETA: 13:5 - ETA: 13:4 - ETA: 13:3 - ETA: 13:1 - ETA: 13

In [87]:
print("Testing size " , str(test_images.shape))
print("Predictions array " , str(preds.shape))

Testing size  (2178, 256, 256, 3)
Predictions array  (2178, 2)


In [88]:
np.set_printoptions(suppress=True,   formatter={'float_kind':'{:0.0f}'.format}) 
# print (preds[:,1])
# print(test_image_names)

In [89]:
from collections import OrderedDict

file_names = OrderedDict([ ('image_id', test_image_names), ('has_oilpalm', np.around(preds[:,1], decimals=0).astype(int)) ])
 
df = pd.DataFrame.from_dict(file_names)
df.to_csv("leaderboard_holdout_data.csv", index = False, sep=',', encoding='utf-8')

len(df)

2178